<a href="https://colab.research.google.com/github/mgontarz20/PhUnwr_CNN/blob/main/PanelSterowaniaGC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Imports


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/BadaniaMchtr')
import numpy as np
import os
import tqdm
from tqdm import tqdm
from keras.losses import MeanSquaredError
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import img_to_array, load_img
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard,CSVLogger,LambdaCallback
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from keras import backend as K
import json


#User Imports

In [ ]:
import genConfig
import plotter
import QPI_DIP_Net

# Definition of hyperparameters

In [ ]:
test_size = 0.25
random_state = 10
n_filters = 128
stop_patience = 11
batch_size = 8
epoch_limit = 200
norm = False
coeff = 1.0
if norm:
    coeff = 255.0
activation_function = 'relu'
state = 'new'
metrics = ['accuracy']

#Name definition



In [ ]:
date = datetime.now().strftime("%m-%d-%Y_%H-%M-%S")
root = '/content/drive/MyDrive/BadaniaMchtr/'
pathtoDataSet = f"{root}Datasets/"
dataset = "dataset_10_Combined_256x256_10-27-2021_13-06-09"
type1 = "resc_wrpd"
type2 = "resc"
comment = "QPIDPI_MSE-SSIM"
name = f"QPI_DPI_Net_{type1}_{date}_{comment}"

# Path Definition
# Folder Generation

In [ ]:


os.makedirs(f'{root}Results/{name}', exist_ok=True)
os.makedirs(f'{root}Results/{name}/model', exist_ok=True)
os.makedirs(f'{root}Results/{name}/cfg', exist_ok=True)
os.makedirs(f'{root}Results/{name}/csv', exist_ok=True)
os.makedirs(f'{root}Results/{name}/logs', exist_ok=True)
cfg_dir = f'{root}Results/{name}/cfg'

# Custom Functions (Callbacks, Loss)

In [ ]:

def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

def SSIMLoss(y_true, y_pred):
  return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 255.0))

def SSIM_MSE(y_true, y_pred):
  return mean_squared_error(y_true, y_pred)/SSIMLoss(y_true, y_pred)

In [ ]:
callbacks = [
    EarlyStopping(patience=stop_patience, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.0000001, verbose=1),
    ModelCheckpoint(f'{root}Results/{name}/model/{name}.h5', verbose=1, save_best_only=True),
    #Tensordash(ModelName=f"{name}", email='mgontarz15@gmail.com', password='dupadupa'),
    TensorBoard(log_dir=f"{root}Results/{name}/logs", write_graph=True, write_images= True, update_freq=5),
    CSVLogger(f"{root}Results/{name}/csv/{name}.csv"),
    #json_logging_callback,
            ]

#Importing Images

In [ ]:
X = np.load("/content/drive/MyDrive/BadaniaMchtr/Datasets/dataset_10_Combined_256x256_10-27-2021_13-06-09/X_dataset_10.npy")
y = np.load("/content/drive/MyDrive/BadaniaMchtr/Datasets/dataset_10_Combined_256x256_10-27-2021_13-06-09/y_dataset_10.npy")

print(X.shape, y.shape)


(6001, 256, 256, 1) (6001, 256, 256, 1)


In [ ]:
genConfig.write_cfg(cfg_dir, name, 'w', type_input = type1, type_output= type2, dataset = dataset)

del type1
del type2
del dataset

# Train Test Splitting


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)
del X
del y

(4500, 256, 256, 1) (1501, 256, 256, 1) (4500, 256, 256, 1) (1501, 256, 256, 1)


In [ ]:
genConfig.write_cfg(cfg_dir, name, "a", test_size = test_size, random_state = random_state)
del test_size
del random_state

#Model Loading and Initialization

In [ ]:
input_img = Input((256, 256, 1), name='img')
model = QPI_DIP_Net.get_model(input_img)
model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError,  metrics=metrics)
model.summary()

NameError: ignored

In [ ]:
genConfig.write_cfg(cfg_dir,name, 'a', optimizer = "Adam", loss = comment, metrics = metrics, state = state, n_filters = n_filters)
del n_filters
del state


#Model Fitting

In [ ]:
start = datetime.now()
results = model.fit(X_train, y_train, batch_size=batch_size, epochs=epoch_limit, callbacks=callbacks,
                     validation_data=(X_valid, y_valid))

stop = datetime.now()



In [ ]:
time = stop - start
del stop
del start

In [ ]:
genConfig.write_cfg(cfg_dir,name, "a", batch_size = batch_size, epoch_limit = epoch_limit, time = time, activation_function = activation_function)
del epoch_limit
del time

#Plotting Results

In [ ]:
plotter.plot_model_data(model, name, results, root)
genConfig.write_cfg(cfg_dir,name, "a",  best_loss = np.min(results.history["val_loss"]), for_epoch_loss = np.argmin(results.history["val_loss"]), best_acc = np.max(results.history["val_accuracy"]), for_epoch_acc = np.argmax(results.history["val_accuracy"]))
del results

#Model Evaluation

In [ ]:
score = model.evaluate(X_train, y_train,
                            batch_size=batch_size)
labels = model.metrics_names

header_train  = "\n-------TRAIN EVALUATION-------"
for i, loss in enumerate(len(labels)):
  train_loss = f"{loss} : {score[i]}"
  genConfig.write_cfg(cfg_dir,name, "a", _ = header_train, TrL = train_loss)

In [ ]:
score= model.evaluate(X_valid, y_valid,
                            batch_size=batch_size)

header_test = "\n-------TEST EVALUATION-------"

for i, loss in enumerate(len(labels)):
  test_loss = f"{loss} : {score[i]}"
  genConfig.write_cfg(cfg_dir,name, "a", _ = header_train, TeL = test_loss)

del score
del train_loss
del header_test
del header_train
del test_loss
del labels

In [ ]:
!apt-get install rar

In [ ]:
# name = "UNetResNet5lvl_resc_wrpd_10-14-2021_22-11-02_SSIMloss"
# !rar a rf'/content/drive/MyDrive/UNetResNet5lvl_resc_wrpd_10-14-2021_22-11-02_SSIMloss_Results' '/content/drive/MyDrive/BadaniaMchtr/Results/UNetResNet5lvl_resc_wrpd_10-14-2021_22-11-02_SSIMloss'
# from google.colab import files
# files.download("/content/drive/MyDrive/UNetResNet5lvl_resc_wrpd_10-14-2021_22-11-02_SSIMloss_Results.rar")
